In [1]:
# some imports
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
plt.rc('font', size=12) 
plt.rc('figure', figsize = (12, 5))

# Settings for the visualizations
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("notebook", font_scale=1, rc={"lines.linewidth": 2,'font.family': [u'times']})
sns.set_theme(style="darkgrid", context="talk", palette="mako")

import pandas as pd
pd.set_option('display.max_rows', 25)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 50)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")
warnings.filterwarnings("ignore")

# create output folder
if not os.path.exists('output'):
    os.makedirs('output')
if not os.path.exists('output/session1'):
    os.makedirs('output/session1')

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

# --- 1. Cargar Datos ---
try:
    # Ajustando las rutas a las que has proporcionado (aunque la estructura del archivo .csv difiere un poco)
    # Asumo que las rutas completas en Kaggle son las que mencionas en el prompt,
    # pero a menudo los notebooks usan una forma más corta. Uso tus rutas específicas:
    train_df = pd.read_csv('/kaggle/input/ml-ub-2025-project-2/train.csv', index_col=0)
    test_df = pd.read_csv('/kaggle/input/ml-ub-2025-project-2/test_features.csv', index_col=0)
    
    # Revisión de los nombres de los archivos originales del prompt de Kaggle vs las rutas:
    # Archivos originales: train.csv, test_features.csv
    # Rutas proporcionadas: train_set.csv, test_set.csv
    # Usaré las rutas proporcionadas, asumiendo que son las correctas para el entorno.

except FileNotFoundError:
    print("Error: Asegúrate de que las rutas de los archivos son correctas en tu entorno Kaggle.")
    
# --- 2. Preprocesamiento Básico ---
# Convertir la variable objetivo 'loan_status' a numérica (0/1)
le = LabelEncoder()
train_df['loan_status'] = le.fit_transform(train_df['loan_status'])

# Identificar columnas categóricas y numéricas (muy simple)
categorical_cols = train_df.select_dtypes(include=['object']).columns
numerical_cols = train_df.select_dtypes(include=['int64', 'float64']).columns.drop('loan_status')

# Codificación One-Hot para variables categóricas
train_df = pd.get_dummies(train_df, columns=categorical_cols, drop_first=True)
test_df = pd.get_dummies(test_df, columns=categorical_cols, drop_first=True)

# Asegurar que ambos conjuntos tienen las mismas columnas después de OHE (crucial para Kaggle)
train_cols = set(train_df.columns) - set(['loan_status'])
test_cols = set(test_df.columns)
common_features = list(train_cols.intersection(test_cols))

X_train = train_df[common_features]
y_train = train_df['loan_status']
X_test_kaggle = test_df[common_features]

# Imputación de valores faltantes (usando la media para todos los tipos)
imputer = SimpleImputer(strategy='mean')
X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X_train.columns)
X_test_kaggle = pd.DataFrame(imputer.transform(X_test_kaggle), columns=X_test_kaggle.columns)

# Escalado (no siempre necesario para LR, pero buena práctica)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_kaggle_scaled = scaler.transform(X_test_kaggle)


# --- 3. Entrenamiento del Modelo y Predicción ---
# Inicializar y entrenar el modelo de Regresión Logística
model = LogisticRegression(solver='liblinear', random_state=42)
model.fit(X_train_scaled, y_train)

# Predecir las probabilidades para el conjunto de prueba (la probabilidad de la clase 1)
# Esto predice la probabilidad, que asumo es el formato de la columna 'Price'
y_pred_proba = model.predict_proba(X_test_kaggle_scaled)[:, 1]


# --- 4. Generación del Archivo de Sumisión ---
# Crear el DataFrame de salida
df_output = pd.DataFrame(y_pred_proba)
df_output = df_output.reset_index()

# La columna 'index' es el ID de la fila y la columna de predicción se llama 'loan_status'
# (Asumo que el target en 'sample_submission.csv' se llama 'loan_status', basándome en el prompt)
df_output.columns = ['id', 'loan_status'] # Nota: Usé 'loan_status' aquí, ya que el target es binario.

# El prompt indica 'Price', pero el target es 'loan_status'. Me ciño a la convención del target binario.
# Si el archivo sample_submission usa 'Price', simplemente cambia 'loan_status' por 'Price' en la línea anterior.

# Si el formato esperado es:
# df_output.columns = ['index','Price'] # Descomentar esta línea si 'Price' es el nombre de la columna

# Asignar el índice del test set al 'index' de salida
df_output['id'] = test_df.index.values

# Guardar la sumisión
df_output.to_csv('/kaggle/working/submission.csv', index=False)

print("✅ ¡Sumisión inicial generada con éxito!")
print(f"Las primeras 5 filas del archivo submission.csv:")
print(df_output.head())

✅ ¡Sumisión inicial generada con éxito!
Las primeras 5 filas del archivo submission.csv:
      id  loan_status
0  43983     0.135446
1  43984     0.002643
2  43985     0.849973
3  43986     0.000325
4  43987     0.055379
